In [ ]:
# daily kos

from bs4 import BeautifulSoup
import requests as r
import json
import re
import datetime

DKURL = 'http://www.dailykos.com'
bad_articles = []
urls = []
for page in range(1,10420):
    print(page)
    html = BeautifulSoup(r.get(f"{DKURL}/?page={page}").text.encode('ascii',"ignore").decode("ascii"), 'html.parser') 
    articles = html.select("body .container .site-content .row .stories-section .row .col-sm-12 .story")
    for article in articles:
        try:
            a = article.find('a' , class_='see-full-story').get("href")
            url = f"{DKURL}{a}"
            date = re.search(r'\d+/\d+/\d+', a).group(0)
            year, month, day = date.split('/')
            urls.append({'url': url, 'date': {'year': int(year), 'month': int(month), 'day': int(day)}})
        except:
            print("ERROR")
            bad_articles.append({'page': page, 'article': article})

In [44]:
with open('dkos_urls.json', 'r') as file:
    dkos_urls = json.load(file)
len(dkos_urls)

96030

In [185]:
# nypost

import requests
from bs4 import BeautifulSoup
import re
import json
from datetime import datetime

base_url = 'http://nypost.com/tag/politics/page/'
urls = []
for pageNum in range(1,139):
    if pageNum % 5 == 0:
        print("Page:", pageNum)
    r = requests.get(base_url + str(pageNum))
    soup = BeautifulSoup(r.text, 'html.parser')
    articles = soup.find_all(class_='entry-header')
    for article in articles:
        url = article.find('a')['href']
        dateString = article.find('p').text
        raw_date = re.match(r'(.+)\s\|\s', dateString).group(1)
        date = datetime.strptime(raw_date, '%B %d, %Y')
        date = {'year': date.year, 'month': date.month, 'day': date.day}
        urls.append({'url': url, 'date': date})
with open('nypost_urls.json', 'w') as outfile:
    json.dump(urls, outfile)
len(urls)

2298

In [184]:
# townhall

from bs4 import BeautifulSoup
import requests
import re
import sys
import datetime
import json

base_url = "https://townhall.com/columnists/date/"
urls = []
date = "2000/07/16" # date = "2000/07/16"
while date != "2018/12/02":
    print(date)
    html = requests.get(base_url + date).text
    soup = BeautifulSoup(html, 'html.parser')
    content = soup.find('div', class_="col-xs-8").find_all('div', class_='row', recursive=False) # extract content
    for tag in content:
        a_tag = tag.find('div', class_="index-story__title").find('a')
        url = "http://townhall.com" + a_tag["href"]
        title = a_tag.text
        date_start = url.find('20')
        date = url[date_start:date_start+10]
        year, month, day = date.split('/')
        urls.append({'url': url, 'date': {'year': int(year), 'month': int(month), 'day': int(day)}})
    date_formatted = datetime.datetime.strptime(date, "%Y/%m/%d")
    end_date_formatted = date_formatted + datetime.timedelta(days=7)
    date = str(end_date_formatted.strftime("%Y/%m/%d"))
    
for i, arr in enumerate(urls):
    url = arr['url']
    if url.count('http') == 2:
        new_url = url[url.find('http',1):]
        urls[i]['url'] = new_url

with open('townhall_urls.json', 'w') as outfile:
    json.dump(urls, outfile)
    
len(urls)

97760

In [100]:
# breitbart

import json
import datetime
from bs4 import BeautifulSoup
import requests as r

urls = []
BBURL = 'http://www.breitbart.com/big-government'
today = datetime.datetime.today()
numdays = 15 # Go back through /2009/09/10/ 3368
dates = (today - datetime.timedelta(days=x) for x in range(numdays))
for date in dates:
    print(date)
    d = '{}/{}/{}'.format(date.year, date.month, date.day)
    url = '{}/{}'.format(BBURL, d)
    html = BeautifulSoup(r.get(url).text, 'html.parser')
    articles = html.find_all('article')
    for article in articles:
        url = article.find('a')['href']
        if url[0:4] != 'http': # Some links only give a relative link
            # url = 'http://www.breitbart.com' + url
            print("Annoying url")
            continue
        urls.append({'url': url, 'date': {'year': date.year, 'month': date.month, 'day': date.day}})
        if len(urls) % 100 == 0:
            print('{} urls added'.format(len(urls)))
print(len(urls))

2018-11-29 10:39:03.623688
2018-11-28 10:39:03.623688
2018-11-27 10:39:03.623688
2018-11-26 10:39:03.623688
100 urls added
2018-11-25 10:39:03.623688
2018-11-24 10:39:03.623688
2018-11-23 10:39:03.623688
200 urls added
2018-11-22 10:39:03.623688
2018-11-21 10:39:03.623688
2018-11-20 10:39:03.623688
300 urls added
2018-11-19 10:39:03.623688
2018-11-18 10:39:03.623688
2018-11-17 10:39:03.623688
400 urls added
2018-11-16 10:39:03.623688
2018-11-15 10:39:03.623688
494


In [155]:
with open('breitbart_urls.json', 'r') as file:
    breitbart_urls = json.load(file)
len(breitbart_urls)

86034

In [183]:
# ny mag

from bs4 import BeautifulSoup
import requests
import re
import sys
import datetime
import json

# ny mag politics section

base_url = "http://nymag.com/intelligencer/politics/?start=" # remove 0
urls = []
for index in range(5,16950,50):
    html = requests.get(base_url + str(index)).text
    soup = BeautifulSoup(html, 'html.parser')
    try:
        content = soup.find('ol', class_='paginated-feed-list-wrapper').find_all('li', class_='article') # extract content
        print("Scraping at index:", str(index))
        for tag in content:
            date = tag.find('time').text
            month, day, year = date.split('/')
            url = tag.find('a')["href"]
            urls.append({"url": url, "date": {'year': int(year), 'month': int(month), 'day': int(day)}})
    except AttributeError:
        print("Error Scraping at index:", str(index))
        continue

with open('nymag_urls.json', 'w') as outfile:
    json.dump(urls, outfile)
    
len(urls)

15569

In [182]:
# ny mag ideas section

base_url = "http://nymag.com/intelligencer/ideas/?start=" # remove 0
urls = []
for index in range(1,4000,50):
    html = requests.get(base_url + str(index)).text
    soup = BeautifulSoup(html, 'html.parser')
    try:
        content = soup.find('ol', class_='paginated-feed-list-wrapper').find_all('li', class_='article') # extract content
        print("Scraping at index:", str(index))
        for tag in content:
            date = tag.find('time').text
            month, day, year = date.split('/')
            url = tag.find('a')["href"]
            urls.append({"url": url, "date": {'year': int(year), 'month': int(month), 'day': int(day)}})
    except AttributeError:
        print("Error Scraping at index:", str(index))
        continue

with open('nymag_idea_urls.json', 'w') as outfile:
    json.dump(urls, outfile)
    
len(urls)

3761

In [27]:
# mother jones

from bs4 import BeautifulSoup
import requests
import re
import sys
import datetime
import json

base_url = "http://www.motherjones.com/politics/page/"
urls = []

for index in range(1,928): #928
    html = requests.get(base_url + str(index)).text
    soup = BeautifulSoup(html, 'html.parser')
    try:
        content = soup.find_all('ul', class_='articles-list')
        children = sum([c.find_all('li', recursive=False) for c in content], []) # extract content
        for tag in children:
            url = tag.find('h3').find('a')['href'] # store link
            urls.append(url)
    except Exception as e:
        print("Could not print index", index, "because of", str(e))

for i, url in enumerate(urls):
    split = url.split('/')
    date = {'year': int(split[4]), 'month': int(split[5])}
    urls[i] = {'url': url, 'date': date}
    
with open('motherjones_urls.json', 'w') as outfile:
    json.dump(urls, outfile)

len(urls)

18534

In [50]:
with open('dkos_urls.json', 'r') as file:
    dkos_urls = json.load(file)
with open('motherjones_urls.json', 'r') as file:
    motherjones_urls = json.load(file)
with open('nymag_urls.json', 'r') as file:
    nymag_urls = json.load(file)
with open('nymag_idea_urls.json', 'r') as file:
    nymag_idea_urls = json.load(file)
with open('nypost_urls.json', 'r') as file:
    nypost_urls = json.load(file)
with open('breitbart_urls.json', 'r') as file:
    breitbart_urls = json.load(file)
with open('townhall_urls.json', 'r') as file:
    townhall_urls = json.load(file)
print(len(dkos_urls))
print(len(motherjones_urls))
print(len(nymag_urls))
print(len(nymag_idea_urls))
print(len(nypost_urls))
print(len(breitbart_urls))
print(len(townhall_urls))

96030
18534
15569
3761
2298
86034
97760


In [55]:
from newspaper import Article
def parse(data, dest):
    articles = []
    bad_articles = []
    for article in data:
        if len(articles) % 20 == 0:
            print(len(articles))
        try:
            url = article['url']
            a = Article(url)
            a.download()
            a.parse()
            article['text'] = a.text
            articles.append(article)
        except Exception as e:
            bad_articles.append(article)
            print(e)
            pass
    print(str(len(articles)), "articles parsed out of", str(len(data)))
    with open(dest, 'w') as outfile:
        json.dump(articles, outfile)
    return articles, bad_articles

In [56]:
articles, bad_articles = parse(nypost_urls, "nypost_data.json")

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
You must `download()` an article first!

1540
You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

1560
You must `download()` an article first!

1560
You must `download()` an article first!

1560
You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article fir

In [69]:
articles, bad_articles = parse(nymag_idea_urls, "nymag_idea_data.json")

0
20
You must `download()` an article first!

40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2820
2840
2860
2880
2900
2920
2940
2960
2980
3000
3020
3040
3060
3080
3100
3120
3140
3160
3180
3200
3220
3240
3260
3280
3300
3320
3340
3360
3380
3400
3420
3440
3460
3480
3500
3520
3540
3560
3580
3600
3620
3640
3660
3680
3700
3720
3740
3760 articles parsed out of 3761


In [81]:
articles, bad_articles = parse(nymag_urls, "nymag_politics_data.json")

0
You must `download()` an article first!

You must `download()` an article first!

20
You must `download()` an article first!

You must `download()` an article first!

40
You must `download()` an article first!

60
80
You must `download()` an article first!

You must `download()` an article first!

100
You must `download()` an article first!

You must `download()` an article first!

120
You must `download()` an article first!

140
You must `download()` an article first!

You must `download()` an article first!

160
You must `download()` an article first!

You must `download()` an article first!

180
You must `download()` an article first!

You must `download()` an article first!

200
You must `download()` an article first!

You must `download()` an article first!

220
You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

240
260
You must `download()` an article first!

260
280

In [100]:
articles, bad_articles = parse(motherjones_urls, "motherjones_data.json")

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2820
2840
2860
2880
2900
2920
2940
2960
2980
3000
3020
3040
3060
3080
3100
3120
3140
3160
3180
3200
3220
3240
3260
3280
3300
3320
3340
3360
3380
3400
3420
3440
3460
3480
3500
3520
3540
3560
3580
3600
3620
3640
3660
3680
3700
3720
3740
3760
3780
3800
3820
3840
3860
3880
3900
3920
3940
3960
3980
4000
4020
4040
4060
4080
4100
4120
4140
You must `downlo

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `download()` an article first!

You must `downlo

11980
12000
12020
12040
12060
12080
12100
12120
12140
12160
12180
12200
12220
12240
12260
12280
12300
12320
12340
12360
12380
12400
12420
12440
12460
12480
12500
12520
12540
12560
12580
12600
12620
12640
12660
12680
12700
12720
12740
12760
12780
12800
12820
12840
12860
12880
12900
12920
12940
12960
12980
13000
13020
13040
13060
13080
13100
13120
13140
13160
13180
13200
13220
13240
13260
13280
13300
13320
13340
13360
13380
13400
13420
13440
13460
13480
13500
13520
13540
13560
13580
13600
13620
13640
13660
13680
13700
13720
13740
13760
13780
13800
13820
13840
13860
13880
13900
13920
13940
13960
13980
14000
14020
14040
14060
14080
14100
14120
14140
14160
14180
14200
14220
14240
14260
14280
14300
14320
14340
14360
14380
14400
14420
14440
14460
14480
14500
14520
14540
14560
14580
14600
14620
14640
14660
14680
14700
14720
14740
14760
14780
14800
14820
14840
14860
14880
14900
14920
14940
14960
14980
15000
You must `download()` an article first!

15020
15040
15060
You must `download()` an arti

In [2]:
import json
from newspaper import Article

In [2]:
with open('townhall_data.json', 'r') as file:
    articles = json.load(file)
with open('townhall_bad_data.json', 'r') as file:
    bad_articles = json.load(file)

In [3]:
len(articles)

97509

In [4]:
len(bad_articles)

251

In [14]:
articles2 = []
bad_articles2 = []
for article in bad_articles:
    try:
        url = article['url']
        a = Article(url)
        a.download()
        a.parse()
        new_article = article.copy()
        new_article['text'] = a.text
        articles2.append(new_article)
    except Exception as e:
        bad_articles2.append(article)
        print(e)

Article `download()` failed with 400 Client Error: Bad Request for url: https://townhall.com/columnists/anncoulter/2000/09/15/bush_can_name_the_a**hole on URL https://townhall.com/columnists/anncoulter/2000/09/15/bush_can_name_the_a**hole

Article `download()` failed with 400 Client Error: Bad Request for url: https://townhall.com/columnists/johnleo/2000/12/27/me**y_chr***mas on URL https://townhall.com/columnists/johnleo/2000/12/27/me**y_chr***mas

Article `download()` failed with 400 Client Error: Bad Request for url: https://townhall.com/columnists/williamfbuckley/2001/01/18/pardon_clinton_!*! on URL https://townhall.com/columnists/williamfbuckley/2001/01/18/pardon_clinton_!*!

Article `download()` failed with 400 Client Error: Bad Request for url: https://townhall.com/columnists/johnleo/2002/12/23/merry_c*********s_to_all! on URL https://townhall.com/columnists/johnleo/2002/12/23/merry_c*********s_to_all!

Article `download()` failed with 400 Client Error: Bad Request for url: http

In [15]:
len(articles2), len(bad_articles2)

(227, 24)

In [26]:
sorted([len(article['text']) for article in articles2])

[118,
 118,
 118,
 118,
 118,
 193,
 1599,
 2027,
 2650,
 2668,
 2683,
 2787,
 2845,
 2906,
 2912,
 2945,
 2970,
 3061,
 3140,
 3249,
 3256,
 3261,
 3277,
 3366,
 3369,
 3394,
 3426,
 3507,
 3513,
 3535,
 3543,
 3561,
 3571,
 3586,
 3601,
 3619,
 3632,
 3633,
 3646,
 3651,
 3654,
 3722,
 3754,
 3754,
 3761,
 3797,
 3807,
 3811,
 3834,
 3836,
 3885,
 3888,
 3889,
 3900,
 3917,
 3937,
 3957,
 3965,
 3987,
 4010,
 4013,
 4030,
 4040,
 4058,
 4068,
 4074,
 4079,
 4082,
 4086,
 4087,
 4097,
 4125,
 4151,
 4163,
 4175,
 4177,
 4178,
 4197,
 4201,
 4205,
 4214,
 4216,
 4239,
 4242,
 4256,
 4260,
 4261,
 4265,
 4279,
 4285,
 4286,
 4295,
 4297,
 4297,
 4304,
 4319,
 4338,
 4341,
 4350,
 4351,
 4360,
 4360,
 4376,
 4380,
 4422,
 4423,
 4439,
 4450,
 4466,
 4474,
 4493,
 4499,
 4501,
 4504,
 4506,
 4511,
 4514,
 4534,
 4551,
 4556,
 4559,
 4561,
 4594,
 4605,
 4608,
 4614,
 4614,
 4616,
 4616,
 4620,
 4625,
 4626,
 4635,
 4676,
 4677,
 4677,
 4686,
 4700,
 4701,
 4710,
 4711,
 4714,
 4729,
 4742

In [27]:
for article in articles2:
    if len(article['text']) == 193:
        print(article)

{'url': 'https://townhall.com/columnists/richgalen/2006/02/06/connecting_the_dots_to_iran', 'date': {'year': 2006, 'month': 2, 'day': 6}, 'text': 'The Only Good Republican Is A Dead Republican\n\n“The vote in Vienna was promoted as a significant victory for the Bush administration …”\n\nWith no explanation as to who was doing the “promoting.”'}


In [31]:
articles2 = len([len(article['text']) for article in articles2 if len(article['text']) > 193])

221

In [33]:
[len(article['text']) for article in articles2 if len(article['text']) <= 193]

[118, 118, 118, 193, 118, 118]

In [36]:
for article in articles2:
    if len(article['text']) > 193:
        articles.append(article)

In [37]:
len(articles)

97730

In [39]:
articles[-5:]

[{'url': 'http://townhall.com/columnists/calvinbeisner/2018/06/21/new-yorker-writer-lashes-out-at-scott-pruitts-evangelical-supporters-n2492800',
  'date': {'year': 2018, 'month': 6, 'day': 21},
  'text': 'The opinions expressed by columnists are their own and do not represent the views of Townhall.com.\n\n“How is it that the administrator of the Environmental Protection Agency, Scott Pruitt, a public servant who walks around clouded in ethics scandals like Pig-Pen in his swirl of dust, still holds his job?”\n\nThat snide lead by Margaret Talbot to her New Yorker article June 8 titled “Are Evangelical Leaders Saving Scott Pruitt’s Job?” should tip off even the uninitiated that what follows will be—shall we say, something less than journalistic fairness?\n\nTalbot attacks Pruitt, then turns to evangelical leaders whose support she thinks has kept him in office despite his “ethics scandals.”\n\nWhat’s her beef with Pruitt? Well, according to her, he\n\nhas engaged in acts that range from

In [42]:
with open('townhall_data.json', 'w') as outfile:
    json.dump(articles, outfile)

In [45]:
with open('townhall_data.json', 'r') as file:
    articles3 = json.load(file)

In [46]:
print(len(articles))

97730


In [70]:
with open('breitbart_data.json', 'r') as file:
    articles = json.load(file)
with open('breitbart_bad_data.json', 'r') as file:
    bad_articles = json.load(file)

In [111]:
from collections import Counter
counts = Counter([len(article['text']) for article in breitbart])
sorted(counts.items(), key = lambda pair: pair[0])

[(20, 1),
 (26, 1),
 (29, 1),
 (30, 1),
 (33, 2),
 (35, 1),
 (36, 2),
 (37, 3),
 (38, 1),
 (39, 1),
 (40, 1),
 (42, 1),
 (43, 2),
 (44, 1),
 (45, 1),
 (47, 2),
 (49, 1),
 (50, 1),
 (51, 2),
 (52, 6),
 (53, 2),
 (54, 2),
 (55, 5),
 (56, 4),
 (58, 6),
 (59, 1),
 (60, 3),
 (61, 10),
 (62, 3),
 (63, 3),
 (64, 4),
 (65, 6),
 (66, 2),
 (67, 4),
 (68, 4),
 (69, 8),
 (70, 2),
 (71, 5),
 (72, 7),
 (73, 2),
 (74, 8),
 (75, 5),
 (76, 2),
 (77, 4),
 (78, 3),
 (79, 3),
 (80, 3),
 (81, 4),
 (82, 4),
 (83, 4),
 (84, 3),
 (85, 3),
 (86, 6),
 (87, 4),
 (88, 5),
 (89, 5),
 (90, 7),
 (91, 4),
 (92, 7),
 (93, 8),
 (94, 4),
 (95, 10),
 (96, 4),
 (97, 5),
 (98, 6),
 (99, 4),
 (100, 7),
 (101, 1),
 (102, 3),
 (103, 12),
 (104, 5),
 (105, 4),
 (106, 5),
 (107, 1),
 (108, 2),
 (109, 3),
 (110, 1),
 (111, 6),
 (112, 3),
 (113, 4),
 (114, 2),
 (115, 3),
 (116, 3),
 (117, 9),
 (118, 2),
 (119, 5),
 (120, 5),
 (121, 3),
 (122, 3),
 (123, 4),
 (124, 6),
 (125, 5),
 (126, 5),
 (127, 3),
 (128, 8),
 (129, 8),
 (130, 

In [92]:
articles5 = []
bad_articles5 = []
for article in bad_articles4:
    try:
        url = article['url']
        a = Article(url)
        a.download()
        a.parse()
        new_article = article.copy()
        new_article['text'] = a.text
        articles5.append(new_article)
        print(len(articles5))
    except Exception as e:
        bad_articles5.append(article)
        print(e)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [98]:
len(articles), len(bad_articles)

(85631, 403)

In [90]:
len(articles2), len(bad_articles2)

(303, 100)

In [88]:
len(articles3), len(bad_articles3)

(11, 89)

In [91]:
len(articles4), len(bad_articles4)

(29, 60)

In [99]:
len(articles5), len(bad_articles5)

(60, 0)

In [100]:
breitbart = articles + articles2 + articles3 + articles4 + articles5
len(breitbart)

86034

In [109]:
breitbart = [article for article in breitbart if len(article['text']) > 0]
len(breitbart)

84909

In [112]:
with open('breitbart_data.json', 'w') as outfile:
    json.dump(breitbart, outfile)

In [110]:
counts = Counter([len(article['text']) for article in breitbart])
counts

Counter({859: 26,
         3721: 14,
         3004: 12,
         1934: 28,
         1995: 23,
         2576: 19,
         675: 22,
         1733: 25,
         1197: 28,
         1057: 28,
         3159: 15,
         1172: 25,
         1463: 24,
         4808: 5,
         2962: 18,
         928: 22,
         4998: 5,
         2070: 33,
         530: 10,
         1240: 29,
         2245: 22,
         1063: 32,
         901: 27,
         7788: 3,
         3216: 15,
         1402: 32,
         2588: 14,
         3700: 11,
         1488: 24,
         5268: 5,
         3277: 7,
         1869: 24,
         6157: 5,
         2661: 15,
         2201: 18,
         6383: 5,
         1862: 15,
         2821: 13,
         6232: 1,
         5302: 5,
         5462: 7,
         1940: 28,
         2478: 20,
         4290: 7,
         3786: 8,
         2147: 20,
         2937: 16,
         1587: 35,
         2053: 19,
         4004: 5,
         2303: 18,
         5248: 7,
         749: 15,
         1078

In [31]:
source = "nypost"

with open(source + '_data.json', 'r') as file:
    nypost_articles = json.load(file)

In [45]:
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.util import ngrams
import string
import re
import json
import pickle

stopword_set = set()
file = open("stop.txt", 'r')
for line in file.readlines():
    word = line[:line.find('|')] if '|' in line else line
    word = word.strip()
    if len(word) > 0:
        stopword_set.add(word)
file.close()

procedural = set()
file = open("vocabulary/procedural.txt", 'r')
next(file)
for line in file.readlines():
    phrase = line[:line.find('|')] if '|' in line else line
    word1, word2 = phrase.split(" ")
    bigram = (word1, word2)
    procedural.add(bigram)
file.close()

stemmer = SnowballStemmer("english")

def preprocess(text):
    text = text.encode("ascii", "ignore") # remove all non-ascii characters
    text = text.decode("ascii")
    text = text.rstrip().lower()
    text = text.replace('\n', ' ').replace('\t', ' ')
    text = text.replace('-', '').replace('\'', '') # delete hyphens and apostrophes
    text = re.sub('[' + string.punctuation + ']', ' ', text) # replace other punctuation with spaces
    tokens = word_tokenize(text) # convert text to tokens
    tokens = [str(stemmer.stem(word)) for word in tokens if word not in stopword_set] # stem using Porter2
    bigrams = []
    for bigram in ngrams(tokens,2):
        # ignore procedural bigrams and purely numerical bigrams
        if bigram in procedural or bigram[0].isdigit() and bigram[1].isdigit():
            continue
        bigrams.append(bigram)
    return bigrams

In [52]:
# for article in nypost_articles[:100]:
#     text = article['text']
#     bigrams = preprocess(text)
# #     print(bigrams)
bigram_articles = []
for article in nypost_articles:
    bigram_article = article.copy()
    text = bigram_article.pop('text')
    bigrams = preprocess(text)
    bigram_article['bigram'] = bigrams
    bigram_articles.append(bigram_article)

In [64]:
with open('motherjones_data.json', 'r') as file:
    motherjones = json.load(file)

In [65]:
len(motherjones)

18534

In [68]:
for article in motherjones[:5]:
#     print(article)
    text = article['text']
    print(preprocess(text))

[('look', 'news'), ('news', 'can'), ('can', 'trust'), ('trust', 'subscrib'), ('subscrib', 'free'), ('free', 'newslett'), ('newslett', '41st'), ('41st', 'presid'), ('state', 'georg'), ('georg', 'h'), ('h', 'w'), ('w', 'bush'), ('bush', 'die'), ('die', 'friday'), ('friday', 'age'), ('age', '94'), ('94', 'world'), ('world', 'leader'), ('leader', 'direct'), ('direct', 'mention'), ('mention', 'presid'), ('presid', 'donald'), ('donald', 'trump'), ('trump', 'danger'), ('danger', 'isolationist'), ('isolationist', 'polici'), ('polici', 'flood'), ('flood', 'statement'), ('statement', 'imposs'), ('imposs', 'rememb'), ('rememb', 'bush'), ('bush', 'word'), ('call', 'kinder'), ('kinder', 'gentler'), ('gentler', 'nation'), ('nation', 'read'), ('read', 'memori'), ('memori', 'busha'), ('busha', 'veteran'), ('veteran', 'world'), ('world', 'war'), ('war', 'ii'), ('ii', 'ambassador'), ('ambassador', 'unit'), ('unit', 'nation'), ('nation', 'director'), ('director', 'cia'), ('cia', 'envoy'), ('envoy', 'chin

In [113]:
with open('townhall_bigrams.p', 'rb') as file:
    townhall = pickle.load(file)

In [117]:
for article in townhall:
    print(len(article['bigram']))

413
380
345
395
386
399
417
353
455
309
324
422
522
403
414
387
397
357
418
473
463
355
441
799
352
389
441
422
394
416
303
369
359
442
292
321
556
374
385
426
387
318
464
353
522
376
413
359
337
496
442
325
438
474
362
284
424
380
349
430
442
389
365
423
467
352
340
391
396
395
307
381
389
355
180
487
343
388
338
338
388
379
379
400
382
384
367
399
481
391
472
413
491
332
385
302
494
392
342
452
515
357
452
409
363
382
413
387
329
448
369
376
343
357
456
407
396
396
494
387
401
323
285
305
346
357
312
404
392
359
408
401
405
433
337
392
501
457
453
394
324
340
370
381
450
376
455
321
377
330
472
330
458
356
462
279
401
365
398
340
445
403
338
469
448
442
389
300
450
396
399
361
467
367
187
472
322
331
348
505
401
382
378
392
492
367
380
390
441
464
355
342
333
395
405
338
345
445
442
332
479
382
389
437
429
358
398
389
385
368
458
377
501
406
432
382
305
418
472
35
379
464
345
464
421
439
392
392
362
464
368
384
351
408
474
386
382
549
475
372
441
387
342
411
379
559
413
365
412
435
3

441
610
399
388
345
427
394
309
338
446
443
348
709
366
368
341
414
310
408
401
314
407
337
401
512
318
396
376
456
303
388
369
351
370
357
425
477
305
492
351
413
633
352
412
484
386
403
382
408
444
385
595
634
404
371
394
350
427
355
416
338
293
435
367
344
372
343
385
506
329
423
337
285
428
349
453
281
339
405
384
435
391
389
431
433
370
441
406
640
457
410
363
375
421
373
473
412
402
353
359
415
397
422
359
437
385
403
393
924
394
395
379
367
326
288
515
355
476
424
403
355
436
331
442
394
422
432
375
413
432
474
373
389
502
355
398
550
352
475
326
381
472
397
291
400
439
428
303
279
393
407
353
427
378
445
324
416
483
421
442
279
325
376
456
336
375
420
396
424
374
417
325
298
407
459
349
392
414
336
390
374
493
363
319
408
446
300
465
413
434
415
435
435
382
461
424
423
297
491
439
367
440
451
328
390
382
440
412
347
342
394
357
368
327
434
260
395
470
432
393
328
411
397
339
386
626
449
406
295
382
464
381
483
412
345
437
405
518
416
501
304
418
446
427
468
381
430
299
280
440


508
357
491
334
395
393
405
358
404
748
397
357
323
420
440
402
371
457
476
549
331
226
424
524
359
487
306
345
371
400
351
294
414
215
344
345
398
384
342
337
400
358
333
432
399
332
418
391
345
404
504
429
343
293
445
283
419
434
424
342
343
392
386
324
410
478
414
452
361
318
457
404
372
423
455
368
353
453
351
380
403
417
435
333
446
310
428
412
344
407
388
327
401
390
451
437
350
334
510
413
338
437
140
524
560
356
393
380
494
282
414
464
423
411
450
386
359
118
298
391
410
364
294
376
631
539
465
291
350
412
330
362
477
443
502
555
390
468
378
297
478
419
413
337
317
468
373
400
454
361
409
448
396
381
359
382
360
379
410
681
333
426
345
378
539
355
430
379
458
169
416
418
470
247
332
411
349
295
400
350
390
487
352
399
382
796
331
320
357
385
438
596
487
368
394
447
410
424
429
413
395
377
388
409
429
386
396
427
463
431
548
424
324
370
443
679
458
381
374
452
481
318
414
295
631
378
322
327
420
392
452
335
320
475
373
821
401
384
429
458
381
412
451
540
412
378
355
377
454
366


464
313
431
620
315
374
409
407
467
417
392
337
394
432
450
589
415
599
437
398
442
465
474
442
380
447
471
429
385
369
848
441
480
372
371
392
381
482
392
357
407
332
461
397
424
397
390
355
339
426
352
466
332
342
1165
525
450
425
265
438
442
407
387
347
573
433
707
290
403
314
405
433
426
380
389
418
382
414
388
489
401
386
459
435
293
575
436
358
316
396
169
397
491
371
333
489
330
348
446
416
393
420
458
401
440
427
399
343
373
391
435
506
355
1286
445
740
404
426
424
432
345
434
502
430
341
415
589
415
599
796
459
464
404
444
450
456
399
358
516
448
425
298
442
409
356
450
532
351
494
415
368
424
398
360
413
341
736
485
357
417
318
435
426
478
370
386
453
420
395
385
381
430
458
342
423
854
614
444
373
447
351
400
433
421
391
428
448
315
352
466
707
290
403
544
351
431
382
362
441
445
382
382
467
514
478
398
323
416
453
420
349
591
394
342
349
456
488
587
449
360
330
363
383
398
435
353
411
380
390
418
10
339
392
431
389
442
390
530
418
373
452
413
378
933
433
447
415
309
509
404

363
816
433
350
517
395
452
436
368
453
465
475
350
403
374
457
395
453
425
429
388
372
504
436
1159
462
398
350
393
971
409
393
433
545
380
358
278
403
549
460
447
377
399
429
481
771
431
397
429
450
416
374
481
378
391
497
403
329
458
456
520
466
423
346
429
374
498
405
763
420
411
400
422
424
383
326
290
429
369
315
430
398
375
375
411
1249
384
336
369
426
483
475
415
391
537
364
477
409
435
450
399
395
439
333
958
395
359
422
432
387
538
396
362
347
395
460
416
284
443
412
387
416
757
416
379
696
474
416
359
429
457
288
1330
459
410
344
421
428
340
325
368
401
404
411
268
1224
430
327
383
435
398
459
446
387
415
449
429
447
429
399
377
710
492
901
423
441
357
390
437
476
402
422
684
495
451
443
410
331
513
378
339
360
601
341
638
401
326
404
361
455
450
483
453
366
454
421
269
431
450
359
391
386
389
368
370
384
763
436
419
411
432
443
439
403
450
399
395
439
420
401
420
550
965
362
442
337
476
477
407
356
517
526
350
390
429
435
510
508
417
445
393
296
848
424
688
415
676
390
340


...

435
484
522
409
789
664
58
432
355
216
500
569
1033
475
396
331
495
441
281
459
406
451
366
472
480
380
451
410
529
385
419
402
187
325
301
424
503
375
338
466
319
398
392
345
859
381
417
536
479
364
196
386
544
414
365
298
632
499
551
514
429
374
375
339
377
385
394
416
516
404
372
425
166
327
335
306
190
325
531
488
405
439
506
416
425
457
388
269
500
469
341
699
754
307
567
476
436
330
374
386
329
471
342
1286
309
385
467
549
743
245
552
481
850
308
498
426
436
391
610
431
479
376
500
383
355
888
384
409
438
350
423
502
479
495
957
521
381
468
408
352
396
420
406
306
432
451
416
428
478
328
515
392
376
445
547
590
419
432
93
395
428
299
357
416
509
472
303
494
311
404
288
375
337
469
398
406
459
581
416
591
247
517
381
417
355
366
564
329
351
465
413
419
404
526
426
397
368
271
513
327
873
395
554
282
485
583
493
395
377
375
357
378
460
398
365
410
445
288
591
415
349
417
477
371
324
498
353
468
324
993
367
324
589
432
305
249
476
506
739
410
390
455
553
388
460
157
460
424
491
470


539
347
396
578
477
398
444
83
491
573
386
718
410
495
556
476
419
353
476
476
414
465
407
333
561
503
389
356
461
397
545
363
505
544
420
285
533
424
382
589
365
334
390
426
419
418
133
382
394
359
399
381
443
392
419
425
347
568
607
344
388
486
355
498
491
336
468
370
297
502
365
683
422
749
508
93
564
325
342
542
558
291
509
506
674
484
520
556
422
579
367
389
464
394
359
663
435
477
615
801
702
374
697
416
388
588
476
721
342
368
448
481
455
438
439
469
536
418
526
363
494
365
412
328
444
91
325
557
410
429
418
502
455
407
320
471
500
307
466
401
403
503
388
343
451
435
569
444
651
78
469
523
432
271
573
342
432
378
409
466
370
342
412
451
418
712
397
592
401
503
351
548
409
431
348
666
336
545
354
465
475
345
361
412
457
480
468
326
449
327
301
307
207
501
500
359
453
518
382
404
383
449
590
630
642
376
858
350
401
415
394
427
649
511
392
484
784
385
113
721
344
434
422
410
316
398
400
575
406
453
477
539
378
421
503
286
400
496
459
315
303
569
529
374
486
466
278
381
466
464
390


377
480
393
332
505
822
354
503
465
407
344
527
708
494
567
851
570
503
583
341
379
437
84
508
685
478
359
816
428
597
571
366
366
613
561
330
580
512
445
812
506
529
405
697
358
445
267
313
445
428
445
443
288
418
395
405
524
374
493
417
320
607
421
82
396
611
440
477
336
652
366
456
434
266
77
296
330
291
409
403
370
372
402
427
470
479
412
493
542
459
453
610
238
308
457
385
333
390
473
396
334
89
397
379
368
362
652
745
393
76
356
298
366
409
877
449
440
335
433
446
377
306
474
444
456
322
632
317
391
80
67
283
292
460
399
285
445
547
526
420
521
377
364
703
422
534
634
485
380
425
570
489
409
381
380
337
800
423
241
450
607
513
455
429
384
546
405
430
322
321
503
292
543
605
455
419
391
55
399
347
313
329
411
459
470
506
292
380
510
437
437
364
424
458
507
422
402
491
634
569
97
542
629
486
536
157
339
640
434
388
464
358
361
414
82
399
351
393
384
427
387
322
405
669
404
284
386
404
369
428
304
327
290
554
382
446
448
416
628
531
432
598
495
296
286
260
148
709
621
575
575
465
57

364
357
398
424
459
467
664
461
363
459
345
470
420
756
345
480
405
372
574
334
336
392
494
417
375
363
452
445
379
425
477
557
526
331
393
501
357
441
438
365
375
355
515
245
561
693
415
451
603
508
542
538
422
291
402
659
548
636
926
909
459
474
322
295
379
384
337
346
453
414
358
458
328
482
390
465
328
411
470
376
415
537
464
316
435
718
498
667
415
626
457
427
486
462
317
349
477
414
357
407
421
455
364
540
375
400
607
400
213
314
303
519
412
382
341
353
392
417
453
659
626
404
463
368
437
346
448
482
368
568
406
441
493
494
499
385
416
327
419
459
402
680
511
434
366
293
588
476
468
377
485
562
493
918
466
630
572
547
886
397
469
439
722
497
788
358
533
408
468
561
459
430
448
351
523
395
521
456
232
453
440
393
514
403
638
382
507
2747
543
454
405
503
542
368
294
350
403
436
357
402
381
528
469
437
476
385
324
271
605
396
425
341
318
384
450
477
456
418
427
427
471
378
313
445
348
534
426
457
395
478
521
361
409
434
539
306
490
360
357
252
426
395
440
595
535
381
526
401
804
537

279
460
320
370
410
598
444
354
419
451
500
438
308
700
624
431
526
304
451
452
533
423
393
520
378
520
593
515
329
433
535
606
621
423
351
377
477
382
299
463
468
357
367
378
475
632
452
467
433
357
445
401
589
561
396
526
389
508
436
464
304
498
387
391
385
353
365
431
417
415
422
422
359
374
492
449
559
455
476
427
432
468
446
497
450
295
400
394
508
422
361
457
465
559
541
410
460
514
537
1048
716
650
569
542
451
439
427
484
563
485
771
450
461
294
751
460
433
549
499
330
452
472
248
485
487
442
288
393
472
445
382
378
468
344
585
553
444
405
339
377
564
550
504
476
332
428
324
505
364
361
379
195
295
464
396
373
475
451
417
261
512
368
338
390
378
506
452
385
558
523
386
355
382
471
434
376
646
444
312
484
362
396
335
455
373
475
508
473
359
268
475
1422
482
542
361
580
485
510
439
432
461
423
1158
344
536
514
660
469
385
510
704
684
476
565
824
571
663
484
460
308
374
519
411
442
431
410
411
356
407
426
361
407
463
524
443
498
488
311
480
467
417
325
431
380
487
404
388
501
517
4

In [118]:
with open('townhall_bigrams2.p', 'rb') as file:
    townhall2 = pickle.load(file)

UnpicklingError: pickle data was truncated

In [29]:
import pickle
from bs4 import BeautifulSoup
import requests
import datetime

In [30]:
articles = pickle.load(open('motherjones_bigrams_new.p','rb'))
len(articles)

18534

In [19]:
listed_dates = {}
failed_urls = []
for article in articles:
    if len(listed_dates) % 50 == 0:
        print(len(listed_dates))
    url = article['url']
    try:
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        listed_dates[url] = soup.find('span', class_="dateline").text
    except:
        print("Failed visiting", url)
        failed_urls.append(url)
listed_dates

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

{'https://www.motherjones.com/politics/2018/12/george-hw-bush-dead-obama-clinton-trump-letter/': 'December 1, 2018 12:01 PM',
 'https://www.motherjones.com/politics/2018/12/rep-raul-grijalva-is-going-to-make-ryan-zinkes-life-hell-in-the-next-congress/': 'December 1, 2018 6:00 AM',
 'https://www.motherjones.com/politics/2018/11/when-the-manafort-plea-deal-blew-up-it-was-good-news-for-oligarchs/': 'November 30, 2018 12:16 PM',
 'https://www.motherjones.com/politics/2018/11/putin-bin-salman-handshake/': 'November 30, 2018 12:06 PM',
 'https://www.motherjones.com/politics/2018/11/fake-putin-twitter-account-fooled-more-than-a-dozen-major-news-outlets/': 'November 30, 2018 11:42 AM',
 'https://www.motherjones.com/politics/2018/11/facebook-civil-rights-audit-sheryl-sandberg-color-of-change-george-soros-definers/': 'November 30, 2018 10:36 AM',
 'https://www.motherjones.com/politics/2018/11/why-has-it-taken-california-so-long-to-count-ballots-because-it-actually-wants-every-vote-to-count/': 'N

In [35]:
for url in failed_urls:
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    listed_dates[url] = soup.find('span', class_="dateline").find('a').text

In [72]:
import json
with open('motherjones_listed_dates.json', 'w') as outfile:
    json.dump(listed_dates, outfile)

In [1]:
import json
listed_dates = json.load(open('motherjones_listed_dates.json', 'r'))

In [2]:
listed_dates

{'https://www.motherjones.com/politics/2018/12/george-hw-bush-dead-obama-clinton-trump-letter/': 'December 1, 2018 12:01 PM',
 'https://www.motherjones.com/politics/2018/12/rep-raul-grijalva-is-going-to-make-ryan-zinkes-life-hell-in-the-next-congress/': 'December 1, 2018 6:00 AM',
 'https://www.motherjones.com/politics/2018/11/when-the-manafort-plea-deal-blew-up-it-was-good-news-for-oligarchs/': 'November 30, 2018 12:16 PM',
 'https://www.motherjones.com/politics/2018/11/putin-bin-salman-handshake/': 'November 30, 2018 12:06 PM',
 'https://www.motherjones.com/politics/2018/11/fake-putin-twitter-account-fooled-more-than-a-dozen-major-news-outlets/': 'November 30, 2018 11:42 AM',
 'https://www.motherjones.com/politics/2018/11/facebook-civil-rights-audit-sheryl-sandberg-color-of-change-george-soros-definers/': 'November 30, 2018 10:36 AM',
 'https://www.motherjones.com/politics/2018/11/why-has-it-taken-california-so-long-to-count-ballots-because-it-actually-wants-every-vote-to-count/': 'N

In [55]:
from collections import Counter
counts = Counter([article['url'] for article in articles])
counts.most_common(29)

[('https://www.motherjones.com/politics/2008/09/how-fix-it-save-middle-class/',
  2),
 ('https://www.motherjones.com/politics/2007/03/reckless-disregard/', 2),
 ('https://www.motherjones.com/politics/2006/03/whos-charge-here-oceans-and-government/',
  2),
 ('https://www.motherjones.com/politics/2006/03/withdrawal-symptoms/', 2),
 ('https://www.motherjones.com/politics/2005/11/editors-note-novemberdecember-2005/',
  2),
 ('https://www.motherjones.com/politics/2005/09/scorched-earth/', 2),
 ('https://www.motherjones.com/politics/2005/01/sleep-sale/', 2),
 ('https://www.motherjones.com/politics/2003/11/campaign-chic/', 2),
 ('https://www.motherjones.com/politics/2003/03/kyrgyzstan-oil/', 2),
 ('https://www.motherjones.com/politics/2003/03/yemen-military/', 2),
 ('https://www.motherjones.com/politics/2002/09/fight-over-coalbed-methane-drilling/',
  2),
 ('https://www.motherjones.com/politics/2002/01/banking-poverty/', 2),
 ('https://www.motherjones.com/politics/1999/11/outfront/', 2),
 ('h

In [33]:
# Explore/fix listed_dates
for url, listed_date in listed_dates.items():
    if len(listed_date.split()) != 5:
#         day = int(listed_date.split()[1][:-1])
#         valid = 1 <= day and day <= 31
#         if not valid:
#             print(url, listed_date)
#         if 'issue' not in listed_date and 'Issue' not in listed_date:
#         if ',' in listed_date:
        print(listed_date)
        article = [a for a in articles if a['url'] == url][0]
#         print(article)
        print(article['date'])

January/February 2019 Issue
{'year': 2018, 'month': 11}
January/February 2019 Issue
{'year': 2018, 'month': 11}
January/February 2019 Issue
{'year': 2018, 'month': 11}
November/December 2018 Issue
{'year': 2018, 'month': 11}
November/December 2018 Issue
{'year': 2018, 'month': 10}
November/December 2018 Issue
{'year': 2018, 'month': 10}
November/December 2018 Issue
{'year': 2018, 'month': 10}
November/December 2018 Issue
{'year': 2018, 'month': 10}
November/December 2018 Issue
{'year': 2018, 'month': 10}
November/December 2018 Issue
{'year': 2018, 'month': 10}
November/December 2018 Issue
{'year': 2018, 'month': 10}
November/December 2018 Issue
{'year': 2018, 'month': 10}
November/December 2018 Issue
{'year': 2018, 'month': 10}
November/December 2018 Issue
{'year': 2018, 'month': 10}
November/December 2018 Issue
{'year': 2018, 'month': 10}
November/December 2018 Issue
{'year': 2018, 'month': 10}
November/December 2018 Issue
{'year': 2018, 'month': 10}
November/December 2018 Issue
{'yea

{'year': 2014, 'month': 8}
July/August 2014 Issue
{'year': 2014, 'month': 8}
July/August 2014
{'year': 2014, 'month': 8}
July/August 2014 Issue
{'year': 2014, 'month': 7}
May/June 2014 Issue
{'year': 2014, 'month': 6}
July/August 2014 Issue
{'year': 2014, 'month': 6}
July/August 2014 Issue
{'year': 2014, 'month': 6}
July/August 2014 Issue
{'year': 2014, 'month': 6}
Updated Mon Jun 30, 2014 10:50 AM EDT | Mon May 26, 2014 10:56 AM EDT | 
{'year': 2014, 'month': 5}
May/June 2014
{'year': 2014, 'month': 5}
May/June 2014 Issue
{'year': 2014, 'month': 5}
May/June 2014 Issue
{'year': 2014, 'month': 4}
March/April 2014
{'year': 2014, 'month': 4}
March/April 2014 Issue
{'year': 2014, 'month': 3}
March/April 2014 Issue
{'year': 2014, 'month': 3}
March/April 2014 Issue
{'year': 2014, 'month': 3}
March/April 2014 Issue
{'year': 2014, 'month': 3}
March/April 2014
{'year': 2014, 'month': 2}
March/April 2014
{'year': 2014, 'month': 2}
March/April 2014
{'year': 2014, 'month': 2}
March/April 2014
{'ye

{'year': 2010, 'month': 9}
November/December 2010 Issue
{'year': 2010, 'month': 9}
November/December 2010 Issue
{'year': 2010, 'month': 9}
September/October 2010 Issue
{'year': 2010, 'month': 9}
November/December 2010 Issue
{'year': 2010, 'month': 9}
September/October 2010 Issue
{'year': 2010, 'month': 9}
September/October 2010 Issue
{'year': 2010, 'month': 9}
September/October 2010 Issue
{'year': 2010, 'month': 9}
September/October 2010 Issue
{'year': 2010, 'month': 9}
September/October 2010 Issue
{'year': 2010, 'month': 9}
September/October 2010 Issue
{'year': 2010, 'month': 9}
September/October 2010 Issue
{'year': 2010, 'month': 9}
July/August 2010 Issue
{'year': 2010, 'month': 8}
July/August 2010 Issue
{'year': 2010, 'month': 8}
July/August 2010 Issue
{'year': 2010, 'month': 8}
July/August 2010 Issue
{'year': 2010, 'month': 8}
July/August 2010 Issue
{'year': 2010, 'month': 8}
May/June 2010 Issue
{'year': 2010, 'month': 7}
September/October 2009
{'year': 2010, 'month': 7}
July/Augus

{'year': 2008, 'month': 10}
November/December 2008 Issue
{'year': 2008, 'month': 10}
November/December 2008 Issue
{'year': 2008, 'month': 10}
September/October 2008 Issue
{'year': 2008, 'month': 10}
September/October 2008 Issue
{'year': 2008, 'month': 10}
September/October 2008 Issue
{'year': 2008, 'month': 9}
September/October 2008 Issue
{'year': 2008, 'month': 9}
September/October 2008 Issue
{'year': 2008, 'month': 9}
September/October 2008 Issue
{'year': 2008, 'month': 9}
September/October 2008 Issue
{'year': 2008, 'month': 9}
September/October 2008 Issue
{'year': 2008, 'month': 9}
September/October 2008 Issue
{'year': 2008, 'month': 9}
September/October 2008 Issue
{'year': 2008, 'month': 9}
September/October 2008 Issue
{'year': 2008, 'month': 9}
September/October 2008 Issue
{'year': 2008, 'month': 9}
September/October 2008 Issue
{'year': 2008, 'month': 9}
September/October 2008 Issue
{'year': 2008, 'month': 9}
September/October 2008 Issue
{'year': 2008, 'month': 9}
September/Octobe

{'year': 2007, 'month': 12}
January/February 2008 Issue
{'year': 2007, 'month': 12}
November/December 2007 Issue
{'year': 2007, 'month': 11}
November/December 2007 Issue
{'year': 2007, 'month': 11}
November/December 2007 Issue
{'year': 2007, 'month': 11}
November/December 2007 Issue
{'year': 2007, 'month': 11}
November/December 2007 Issue
{'year': 2007, 'month': 10}
November/December 2007 Issue
{'year': 2007, 'month': 10}
November/December 2007 Issue
{'year': 2007, 'month': 10}
November/December 2007 Issue
{'year': 2007, 'month': 10}
November/December 2007 Issue
{'year': 2007, 'month': 10}
November/December 2007 Issue
{'year': 2007, 'month': 10}
November/December 2007 Issue
{'year': 2007, 'month': 10}
November/December 2007 Issue
{'year': 2007, 'month': 10}
November/December 2007 Issue
{'year': 2007, 'month': 10}
September/October 2007 Issue
{'year': 2007, 'month': 9}
September/October 2007 Issue
{'year': 2007, 'month': 9}
September/October 2007 Issue
{'year': 2007, 'month': 9}
Septemb

...

{'year': 1998, 'month': 1}
January/February 1998 Issue
{'year': 1998, 'month': 1}
January/February 1998 Issue
{'year': 1998, 'month': 1}
January/February 1998 Issue
{'year': 1998, 'month': 1}
November/December 1997 Issue
{'year': 1997, 'month': 11}
November/December 1997 Issue
{'year': 1997, 'month': 11}
November/December 1997 Issue
{'year': 1997, 'month': 11}
November/December 1997 Issue
{'year': 1997, 'month': 11}
November/December 1997 Issue
{'year': 1997, 'month': 11}
November/December 1997 Issue
{'year': 1997, 'month': 11}
November/December 1997 Issue
{'year': 1997, 'month': 11}
November/December 1997 Issue
{'year': 1997, 'month': 11}
November/December 1997 Issue
{'year': 1997, 'month': 11}
November/December 1997 Issue
{'year': 1997, 'month': 11}
November/December 1997 Issue
{'year': 1997, 'month': 11}
November/December 1997 Issue
{'year': 1997, 'month': 11}
November/December 1997 Issue
{'year': 1997, 'month': 11}
November/December 1997 Issue
{'year': 1997, 'month': 11}
November/D

{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 1996 Issue
{'year': 1996, 'month': 9}
September/October 199

{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
November/December 1995 Issue
{'year': 1995, 'month': 11}
Nov

{'year': 1994, 'month': 11}
November/December 1994 Issue
{'year': 1994, 'month': 11}
November/December 1994 Issue
{'year': 1994, 'month': 11}
November/December 1994 Issue
{'year': 1994, 'month': 11}
November/December 1994 Issue
{'year': 1994, 'month': 11}
November/December 1994 Issue
{'year': 1994, 'month': 11}
November/December 1994 Issue
{'year': 1994, 'month': 11}
November/December 1994 Issue
{'year': 1994, 'month': 11}
November/December 1994 Issue
{'year': 1994, 'month': 11}
November/December 1994 Issue
{'year': 1994, 'month': 11}
November/December 1994 Issue
{'year': 1994, 'month': 11}
November/December 1994 Issue
{'year': 1994, 'month': 11}
September/October 1994 Issue
{'year': 1994, 'month': 9}
September/October 1994 Issue
{'year': 1994, 'month': 9}
September/October 1994 Issue
{'year': 1994, 'month': 9}
September/October 1994 Issue
{'year': 1994, 'month': 9}
September/October 1994 Issue
{'year': 1994, 'month': 9}
September/October 1994 Issue
{'year': 1994, 'month': 9}
September

{'year': 1993, 'month': 9}
September/October 1993 Issue
{'year': 1993, 'month': 9}
September/October 1993 Issue
{'year': 1993, 'month': 9}
September/October 1993 Issue
{'year': 1993, 'month': 9}
September/October 1993 Issue
{'year': 1993, 'month': 9}
September/October 1993 Issue
{'year': 1993, 'month': 9}
September/October 1993 Issue
{'year': 1993, 'month': 9}
September/October 1993 Issue
{'year': 1993, 'month': 9}
July/August 1993 Issue
{'year': 1993, 'month': 7}
July/August 1993 Issue
{'year': 1993, 'month': 7}
July/August 1993 Issue
{'year': 1993, 'month': 7}
July/August 1993 Issue
{'year': 1993, 'month': 7}
July/August 1993 Issue
{'year': 1993, 'month': 7}
July/August 1993 Issue
{'year': 1993, 'month': 7}
July/August 1993 Issue
{'year': 1993, 'month': 7}
July/August 1993 Issue
{'year': 1993, 'month': 7}
July/August 1993 Issue
{'year': 1993, 'month': 7}
July/August 1993 Issue
{'year': 1993, 'month': 7}
July/August 1993 Issue
{'year': 1993, 'month': 7}
July/August 1993 Issue
{'year':

In [51]:
def fix_dates(articles):
    for i, article in enumerate(articles):
        url = articles[i]['url']
        date = articles[i].pop('date')
        listed_date = listed_dates[url]
        date['day'] = int(listed_date.split()[1][:-1]) if len(listed_date.split()) == 5 else 1
        articles[i]['date'] = date
    return articles

In [54]:
bigram_articles = pickle.load(open('motherjones_bigrams_new.p','rb'))
bigram_articles = fix_dates(bigram_articles)
pickle.dump(bigram_articles, open('motherjones_bigrams_new.p', "wb"))

trigram_articles = pickle.load(open('motherjones_trigrams_new.p','rb'))
trigram_articles = fix_dates(trigram_articles)
pickle.dump(trigram_articles, open('motherjones_trigrams_new.p', "wb"))

In [ ]:
for i, article in enumerate(articles):
    listed_date = listed_dates[article['url']]
    day = int(listed_date.split()[1][:-1])
    new_date = article['date'].copy()
    new_date['day'] = day if 1 <= day and day <= 31 else 1
    articles[i]['date'] = new_date
dates